In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Import util packages
import sys
import os

# Add the parent directory to the system path
sys.path.append(os.path.abspath('..'))
from utils import open_file

In [3]:
# Load train and test data
train_data = open_file('cleaned_train.csv')
test_data = open_file('cleaned_test.csv')

/Users/mac/10a/10-academy-week-4/utils/data_utils.py:31: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [4]:
# Display the first 5 rows of the train data
print(train_data.head())

   Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      2          5  2015-07-31   6064        625     1      1            0   
2      3          5  2015-07-31   8314        821     1      1            0   
3      4          5  2015-07-31  13995       1498     1      1            0   
4      5          5  2015-07-31   4822        559     1      1            0   

   SchoolHoliday  Year  Month  Day  Weekday  IsWeekend  
0              1  2015      7   31        4      False  
1              1  2015      7   31        4      False  
2              1  2015      7   31        4      False  
3              1  2015      7   31        4      False  
4              1  2015      7   31        4      False  
